In [24]:
def printState(stateInd, state):
    print("-------------------------------------")
    print("State ", stateInd)
    print("Temperature: ", state['T'], "K")
    print("Pressure: ", state['P'], "Pa")
    print("Density: ", state['Rho'], "kg/m^3")
    print("Abs. Velocity: ", state['c'], "m/s")
    print("Rad. Velocity: ", state['c_r'], "m/s")
    print("Circ. Velocity: ", state['c_u'], "m/s")
    print("Ax. Velocity: ", state['c_z'], "m/s")
    
    
def geometry_swirler(channelNo, vaneAngle, channelHeight, channelRad_1, channelRad_2, outletRad, arcFac1, arcFac2):
    '''Basic geometric parameters of the swirler'''
    channelArcLen_1 = arcFac1 * (2. * math.pi * channelRad_1) / channelNo
    channelArcLen_2 = arcFac2 * (2. * math.pi * channelRad_2) / channelNo
    swirlerGeometry = {'channelNo':  channelNo,
                       'vaneAngle': vaneAngle ,
                       'channelHeight' : channelHeight,
                       'channelRad_1' : channelRad_1, 
                       'channelRad_2' : channelRad_1,
                       'outletRad': outletRad,
                       'channelArcLen_1' : channelArcLen_1,
                       'channelArcLen_2' : channelArcLen_2,
                       'channelArea_1' : channelHeight * channelArcLen_1,
                       'channelArea_2' : channelHeight * channelArcLen_2,
                       'outletArea' : 0.5 * math.pi * outletRad**2 / channelNo 
                        }
    
    print("-------------------------------------")
    print("Number of channels: ", channelNo)
    print("Inner Radius: ", channelRad_2 * 1000, "mm")
    print("Outer Radius: ", channelRad_1 * 1000, "mm")
    print("Channel Height: ", channelHeight * 1000, "mm")
    print("Vane Angle: ", vaneAngle * (180. / math.pi), "°")
    print("Inner Channel Area: %.3f mm" % (swirlerGeometry['channelArea_2'] * 1.0E6))
    print("Outer Channel Area: %.3f mm" % (swirlerGeometry['channelArea_1'] * 1.0E6))
    return swirlerGeometry
    
    
def state_swirlerInlet(T_1, P_1, mflux, thermoData, geom):
    '''Calculate the state at the inlet of the swirler'''
    Ru = thermoData['Ru']
    MW = thermoData['MW']
    
    Rho_1 = (P_1 * MW) / (Ru * T_1)

    # Compute Inlet velocity
    #--------------------------------------------------
    c_1_r = mflux /  (Rho_1 * geom['channelArea_1'])
    c_1_u = 0.0
    c_1   = math.sqrt(c_1_r*c_1_r + c_1_u*c_1_u)

    state_1 = {'Rho': Rho_1, 'T'  : T_1, 'P'  : P_1,
               'c'  : c_1, 'c_r': c_1_r, 'c_u': c_1_u, 'c_z': 0.0}
    
    printState(1, state_1)
    return state_1


def state_swirlerOutlet(mflux, s1, geom, td):
    '''Solve for the state 2 '''
    # Solve Pressure Equation 
    #--------------------------------------------------
    velFac   = (mflux / (s1['Rho'] * geom['channelArea_2']))**2
    angleFac = 0.5 * (1. + math.tan(geom['vaneAngle']) * math.tan(geom['vaneAngle']))
    tempFac  = td['cp'] * s1['T']
    expFac1  = (td['gamma']-1.0) / td['gamma']
    expFac2  = -2.0 / td['gamma']

    A12 = 1.0
    B12 =  angleFac * velFac / tempFac
    C12 = -1.0 - (0.5 * s1['c'] * s1['c']) / tempFac

    import scipy.optimize
    def F(x): 
        return A12 * x**expFac1 + B12 * x**expFac2 + C12

    P_ratio_12 = scipy.optimize.broyden2(F, 1.0, f_tol=1e-10)

    P_2 = P_1 * P_ratio_12
    T_2 = T_1 * P_ratio_12**expFac1
    Rho_2 = (P_2 * MW) / (Ru * T_2)
    c_2_r = mflux / (Rho_2 * geom['channelArea_2'])
    c_2_u = math.tan(geom['vaneAngle']) * c_2_r
    c_2   = math.sqrt(c_2_r*c_2_r + c_2_u*c_2_u)
    
    state_2 = {'Rho': Rho_2, 'T'  : T_2, 'P'  : P_2,
               'c'  : c_2, 'c_r': c_2_r, 'c_u': c_2_u, 'c_z': 0.0}
    printState(2, state_2)
    return state_2


def state_chamberInlet(mflux, s, geom, td):
    '''Solve for state at burning chamber inlet'''
    rad_ratio = (geom['channelRad_2'] / geom['outletRad'])**2
    velFac    = (mflux / (s['Rho'] * geom['outletArea']))**2
    tempFac  = td['cp'] * s['T']
    expFac1  = (td['gamma']-1.0) / td['gamma']
    expFac2  = -2.0 / td['gamma']

    A23 = 1.0
    B23 = 0.5 * velFac / tempFac
    C23 = rad_ratio * (0.5 * s['c_u'] * s['c_u' ]) / tempFac - (0.5 * s['c'] * s['c' ]) / tempFac - 1.0
    
    import scipy.optimize
    def F(x): 
        return A23 * x**expFac1 + B23 * x**expFac2 + C23

    P_ratio_23 = scipy.optimize.broyden2(F, 1.0, f_tol=1e-10)

    P_3 = s['P'] * P_ratio_23
    T_3 = s['T'] * P_ratio_23**expFac1
    Rho_3 = (s['P'] * td['MW']) / (td['Ru'] * s['T'])
    c_3_z = mflux / (Rho_3 * geom['channelArea_2'])
    c_3_u =  (geom['channelRad_2'] / geom['outletRad']) * s['c_u']
    c_3   = math.sqrt(c_3_z*c_3_z + c_3_u*c_3_u)
    
    state_3 = {'Rho': Rho_3, 'T'  : T_3, 'P'  : P_3,
               'c'  : c_3, 'c_r': 0.0, 'c_u': c_3_u, 'c_z': c_3_z}
    printState(3, state_3)
    return state_3

In [27]:
import math

# Swirler - Geometric Parameters
#--------------------------------------------------
channelNo = 4
vaneAngle = 40.0 * (math.pi / 180.)
channelHeight = 10. * 1.0E-3
channelRad_1 = 120. * 1.0E-3
channelRad_2 = 60. * 1.0E-3
outletRad    = 60. * 1.0E-3
arcFac1 = 0.7
arcFac2 = 0.5

swirler_geometry = geometry_swirler(channelNo, 
                                    vaneAngle, 
                                    channelHeight, 
                                    channelRad_1, 
                                    channelRad_2, 
                                    outletRad,
                                    arcFac1, arcFac2)

# Thermodata
#--------------------------------------------------
Ru    = 8.3145 # J / (mol K)
cp    = 1.0E3  # J / (kg K)
MW    = 28.9647 * 1.0E-3  # kg / mol
gamma = 1.4    # - 
thermoData = {'Ru': Ru, 'cp': cp, 'MW': MW, 'gamma':gamma}

# Inlet Boundary Conditions
#--------------------------------------------------
mflux_total = 80. * 1.0E-3  
mflux = mflux_total / swirler_geometry['channelNo']  
T_1   = 300.  
P_1   = 101325.
state_1 = state_swirlerInlet(T_1, P_1, mflux, thermoData, swirler_geometry)

# Conditions at swirler outlet
#--------------------------------------------------
state_2 = state_swirlerOutlet(mflux, state_1, swirler_geometry, thermoData)
delta_p_12 = 100. * ((state_1['P'] - state_2['P']) / state_1['P'])
print(("-------------------------------------"))
print("Pressure Loss 1->2: %.3f Percent" % (delta_p_12))

# Conditions at burning chamber inlet
#--------------------------------------------------
state_3 = state_chamberInlet(mflux, state_2, swirler_geometry, thermoData)
delta_p_13 = 100. * ((state_1['P'] - state_3['P']) / state_1['P'])
print(("-------------------------------------"))
print("Pressure Loss 1->3: %.3f Percent" % (delta_p_13))



-------------------------------------
Number of channels:  4
Inner Radius:  60.0 mm
Outer Radius:  120.0 mm
Channel Height:  10.0 mm
Vane Angle:  40.0 °
Inner Channel Area: 471.239 mm
Outer Channel Area: 1319.469 mm
-------------------------------------
State  1
Temperature:  300.0 K
Pressure:  101325.0 Pa
Density:  1.176598403391665 kg/m^3
Abs. Velocity:  12.882571983869935 m/s
Rad. Velocity:  12.882571983869935 m/s
Circ. Velocity:  0.0 m/s
Ax. Velocity:  0.0 m/s
-------------------------------------
State  2
Temperature:  298.9548439871951 K
Pressure:  100094.86597824346 Pa
Density:  1.166377423257591 kg/m^3
Abs. Velocity:  47.5002387626612 m/s
Rad. Velocity:  36.38729395096129 m/s
Circ. Velocity:  30.53256493379089 m/s
Ax. Velocity:  0.0 m/s
-------------------------------------
Pressure Loss 1->2: 1.214 Percent
-------------------------------------
State  3
Temperature:  298.14394199816337 K
Pressure:  99147.8231647506 Pa
Density:  1.166377423257591 kg/m^3
Abs. Velocity:  71.084353

In [76]:
mflux

20000.0